In this notebook, we are loading an existing model and features saved from unlabeled data from a single folder. Using these features we let the model make predictions for each image. Finally, we save the predictions to csv and json files.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import tensorflow as tf

2024-08-07 09:07:30.445555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 09:07:30.445629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 09:07:30.511155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-07 09:07:30.642280: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 09:07:31.863616: W tensorflow/compiler/tf2

### Load validation set in tensorflow format

In [4]:
def load_dataset(path):
    data = pd.read_pickle(path)
    n_features = data['clip_features'][0].shape[-1]
    class_names = data['y'].unique()
    y = np.array([np.where(class_names == e)[0][0] for e in data['y']])
    x = np.concatenate(data['clip_features'].to_numpy())
    print(f'Loaded {len(x)} instances from {path}.')
    print(f'X shape {x.shape} and y shape {y.shape} with labels:\n{data["y"].value_counts()}.')
    train_ds = tf.data.Dataset.from_tensor_slices((x, y)).batch(16)
    return train_ds, class_names, n_features, data['paths']

path = '../new_data_2024/validation.pk'
validation_ds, class_names, n_features, paths = load_dataset(path)

Loaded 2932 instances from ../new_data_2024/validation.pk.
X shape (2932, 512) and y shape (2932,) with labels:
y
Complex AI    2932
Name: count, dtype: int64.


2024-08-07 09:07:56.824621: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-07 09:07:57.063740: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-07 09:07:57.063802: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-07 09:07:57.068419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-07 09:07:57.068512: I external/local_xla/xla/stream_executor

In [5]:
pd.read_pickle(path)

,paths,y,clip_features
0,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.002533, 0.002254, 0.001475, 0.001476, 0.00..."
1,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001595, 0.003633, 0.002438, 0.001778, 0.00..."
2,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001514, 0.003183, 0.002155, 0.001897, 0.00..."
3,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001799, 0.002172, 0.00215, 0.001761, 0.001..."
4,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001634, 0.001647, 0.002796, 0.001132, 0.00..."
...,...,...,...
2927,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.00221, 0.003422, 0.001049, 0.001612, 0.002..."
2928,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001391, 0.001732, 0.001799, 0.00176, 0.003..."
2929,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001863, 0.001716, 0.001486, 0.002432, 0.00..."
2930,../new_data_2024/images_by_class/validation/Co...,Complex AI,"[[0.001982, 0.00182, 0.001515, 0.001769, 0.002..."


In [6]:
from pathlib import Path
p = Path(r"../robot_or_brain_enriched_and_combined_data/images_by_class/validation")
class_list = [d.name for d in p.iterdir()]

In [7]:
class_list

['Acting or Performing machine',
 'Collaborative or Interactive AI',
 'Complex AI',
 'Learning or recognition machine',
 'Mysterious AI',
 'None of the above',
 'Superior human',
 'Thinking machine']

In [8]:
trues = [class_list[int(y)] for _x, y in validation_ds.unbatch()]

### Print class counts for each split

In [9]:
df([y for y in trues]).value_counts()

Acting or Performing machine    2932
Name: count, dtype: int64

Some of these counts are horrible. The number of examples in 'Learning or recognition machine', 'Superior human' and 'Mysterious AI' are really too small. Also, the 'None of the above' class is very large, although that's not really a problem.

### Load our trained model

In [10]:
import tensorflow as tf

# model = tf.keras.models.load_model('./fine_tuned_model_3m6herki')
# model = tf.keras.models.load_model('./fine_tuned_model_3f0vzk68')
# model = tf.keras.models.load_model('./fine_tuned_model_qdesgan9')
# model = tf.keras.models.load_model('./clip_features_model_kuw3ehqp')
model = tf.keras.models.load_model('./clip_features_model_kf5cnvvi')

### Make predictions

In [11]:
logits = model.predict(validation_ds)
predicted = [class_list[v] for v in np.argmax(logits, 1)]
from pandas import DataFrame as df

print(df(predicted).value_counts())
print(df(trues).value_counts())

  1/184 [..............................] - ETA: 1:20

2024-08-07 09:08:35.608377: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:504] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-12.2
  /usr/local/cuda
  /home/christiaan/anaconda3/envs/robot311/lib/python3.11/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/christiaan/anaconda3/envs/robot311/lib/python3.11/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.


184/184 [==============================] - 1s 2ms/step
None of the above                  1853
Collaborative or Interactive AI     432
Acting or Performing machine        230
Thinking machine                    221
Complex AI                          182
Superior human                       11
Mysterious AI                         3
Name: count, dtype: int64
Acting or Performing machine    2932
Name: count, dtype: int64


In [12]:
predicted[:10]

['Complex AI',
 'None of the above',
 'None of the above',
 'None of the above',
 'None of the above',
 'Collaborative or Interactive AI',
 'Collaborative or Interactive AI',
 'None of the above',
 'Acting or Performing machine',
 'None of the above']

In [13]:
trues[:10]

['Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine',
 'Acting or Performing machine']

In [14]:
paths[:10]

0    ../new_data_2024/images_by_class/validation/Co...
1    ../new_data_2024/images_by_class/validation/Co...
2    ../new_data_2024/images_by_class/validation/Co...
3    ../new_data_2024/images_by_class/validation/Co...
4    ../new_data_2024/images_by_class/validation/Co...
5    ../new_data_2024/images_by_class/validation/Co...
6    ../new_data_2024/images_by_class/validation/Co...
7    ../new_data_2024/images_by_class/validation/Co...
8    ../new_data_2024/images_by_class/validation/Co...
9    ../new_data_2024/images_by_class/validation/Co...
Name: paths, dtype: object

In [15]:
df_predicted = df(predicted).reset_index()
df_trues = df(trues).reset_index()

In [16]:
df_predicted = df_predicted.rename(columns={0:'predictions'})
df_trues = df_trues.rename(columns={0:'folder_validation'})

In [17]:
results = []

In [18]:
for prediction, path in zip(predicted, paths):
    r = {}
    r['prediction'] = prediction
    r['path'] = path
    r['filename'] = path.name
    results.append(r)

In [19]:
results = pd.DataFrame(results)

In [20]:
from IPython.display import display
from ipywidgets import widgets, HBox, VBox, Layout


hboxes = []
for class_name in class_list:    
    label = widgets.Label(class_name, layout=Layout(width='220px'))
    
    image_paths = [str(r[1]['path']) for r in list(results.iterrows()) if r[1]['prediction'] == class_name]
    images = [widgets.Image(value=open(p, 'rb').read(), layout=Layout(width='120px')) for p in image_paths[:5]]
    # images=[widgets.Label(t,layout=Layout(width='220px')) for t in image_paths[:10]]
    hbox = HBox([label] + images, layout=Layout(height='90px'))
    hboxes += [hbox]
vbox = VBox(hboxes)
display(vbox)

In [21]:
results.to_csv('predictions_2024_chris_wsl.csv', index=False)
results.to_pickle('predictions_2024_chris_wsl.pkl')
